In [ ]:
import json
import os
import csv
import glob
import errno
import datetime
import pandas as pd
root_dir = os.path.abspath(os.curdir)
raw_dir = os.path.dirname(root_dir)+"/RawSubmissions/"
ids_dir = os.path.dirname(root_dir)+"/SubmissionIDs/"

fields = ['ID','Author','LinkID','Subreddit','Text','Time','Link']

for year in range(2006,2008):
    print(year)
    submissions = raw_dir+'RS_v2_'+str(year)+'*'
    files = sorted(glob.glob(submissions))
    IDFile = ids_dir+str(year)+'.csv'
    IDs = [x.strip() for x in open(IDFile)]
    outfile = os.path.dirname(root_dir)+'/ExtractedSubmissions/'+str(year)+'.csv'

    os.makedirs(os.path.dirname(outfile), exist_ok=True)
    for file in files:
        try:
            with open(file,encoding="utf-8") as cf: # No need to specify 'r': this is the default.
                
                
                with open(outfile, 'a', encoding="utf-8") as f:
                      writer = csv.writer(f,delimiter=',',lineterminator='\n')
                      for line in cf:
                        data = json.loads(line.strip())
                        theID = data['id']
                        if theID in IDs:
                            
                            cid = 't3_'+data['id']
                            subreddit = data['subreddit']
                            link = data['url']
                            author = data['author']
                            cdate =int(data['created_utc'])
                            thedate = datetime.datetime.utcfromtimestamp(cdate)
                            cformat = thedate.strftime('%Y-%m-%d %H:%M:%S')
                            if 'selftext' in data:
                                txt = data['title']+data['selftext']
                            else:
                                txt = data['title']
                            comment = " ".join(txt.split())
                            writer.writerow([theID,author,cid,subreddit,comment,cformat,link])
                            
        except IOError as exc:
            if exc.errno != errno.EISDIR: # Do not fail if a directory is found, just ignore it.
                raise # Propagate other kinds of IOError.
    df = pd.read_csv(outfile,header=None,names =fields)
    df = df.drop_duplicates(subset='ID', keep='first')
    final = df.set_index('ID').loc[IDs].reset_index()
    os.remove(outfile)
    final.to_csv(outfile,index=False,header=False)
print("Done!")

2006
2007
